nest_asyncio는 Python의 asyncio 라이브러리에서 이벤트 루프를 중첩해서 사용할 수 있도록 도와주는 라이브러리입니다.

기본적으로 Python의 asyncio는 한 번 실행된 이벤트 루프를 다시 실행하려고 하면 오류를 발생시킵니다. 예를 들어, Jupyter Notebook 같은 환경에서는 이미 실행 중인 이벤트 루프가 있기 때문에, 새로운 비동기 작업을 실행하려고 하면 문제가 생길 수 있습니다.

nest_asyncio는 이런 문제를 해결하기 위해 기존 이벤트 루프를 패치하여, 이미 실행 중인 이벤트 루프에서도 추가적인 비동기 작업을 실행할 수 있도록 만들어줍니다.

간단한 비유:
기본 asyncio: "이미 도로에 차가 달리고 있으니, 새로운 차를 추가로 못 올려!"
nest_asyncio: "도로를 확장해서 새로운 차도 추가로 달릴 수 있게 만들었어!"

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def hello():
    return {"message": "Hello World"}

In [24]:
import uvicorn
uvicorn.run(app)

INFO:     Started server process [1594]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1594]


# Database 설정 파일 추가

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./myapi.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

이 코드는 Python의 SQLAlchemy 라이브러리를 사용하여 데이터베이스와 상호작용하기 위한 설정을 정의합니다. SQLAlchemy는 Python에서 데이터베이스 작업을 쉽게 처리할 수 있도록 도와주는 강력한 ORM(Object Relational Mapper) 도구입니다. 아래는 코드의 각 부분에 대한 설명입니다.

---

1. **필요한 모듈 임포트**  
   첫 번째로, `sqlalchemy` 라이브러리에서 필요한 구성 요소를 가져옵니다.  
   - `create_engine`: 데이터베이스와의 연결을 생성하는 데 사용됩니다.  
   - `declarative_base`: ORM에서 데이터베이스 테이블을 Python 클래스와 매핑하기 위한 기본 클래스입니다.  
   - `sessionmaker`: 데이터베이스 세션을 생성하고 관리하는 데 사용됩니다.

---

2. **데이터베이스 URL 설정**  
   `SQLALCHEMY_DATABASE_URL` 변수는 데이터베이스 연결 문자열을 정의합니다. 여기서는 SQLite 데이터베이스를 사용하며, 파일 이름은 `myapi.db`입니다. SQLite는 파일 기반 데이터베이스로, 간단한 프로젝트에 적합합니다.

---

3. **데이터베이스 엔진 생성**  
   `create_engine` 함수는 데이터베이스와의 연결을 처리하는 엔진 객체를 생성합니다.  
   - `SQLALCHEMY_DATABASE_URL`을 사용하여 데이터베이스 위치를 지정합니다.  
   - `connect_args={"check_same_thread": False}`는 SQLite에서 멀티스레드 환경에서도 동일한 연결을 사용할 수 있도록 설정합니다. 이는 SQLite의 기본 동작을 변경하는 옵션입니다.

---

4. **세션 로컬 생성**  
   `SessionLocal`은 데이터베이스 세션을 생성하는 팩토리 함수입니다.  
   - `autocommit=False`: 세션에서 변경 사항을 명시적으로 커밋해야 합니다.  
   - `autoflush=False`: 세션이 자동으로 변경 사항을 데이터베이스에 반영하지 않습니다.  
   - `bind=engine`: 이 세션이 위에서 생성한 데이터베이스 엔진과 연결되도록 설정합니다.

---

5. **기본 클래스 정의**  
   `Base = declarative_base()`는 ORM에서 사용할 기본 클래스를 생성합니다. 이 클래스는 이후 데이터베이스 테이블과 매핑될 모든 모델 클래스의 부모 클래스 역할을 합니다.

---

이 코드는 데이터베이스와 상호작용하기 위한 기본 설정을 정의하며, 이후 데이터베이스 모델을 생성하거나 데이터를 삽입/조회하는 데 사용됩니다. 이 구조는 SQLAlchemy를 사용하는 대부분의 프로젝트에서 공통적으로 사용되는 초기 설정입니다.

In [ ]:
from sqlalchemy import Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import relationship

class Question(Base):
    __tablename__ = "questions"

    id = Column(Integer, primary_key=True)
    subject = Column(String, nullable=False)
    content = Column(Text, nullable=False)
    create_date = Column(DateTime, nullable=False)

class Answer(Base):
    __tablename__ = "answers"

    id = Column(Integer, primary_key=True)
    content = Column(Text, nullable=False)
    create_date = Column(DateTime, nullable=False)
    question_id = Column(Integer, ForeignKey("questions.id"))
    question = relationship("Question", backref="answers")
    

위의 코드는 SQLAlchemy를 사용하여 데이터베이스 테이블과 열(column), 그리고 외래 키(foreign key)를 정의하고 관리하는 데 필요한 클래스와 함수들을 포함하고 있습니다. SQLAlchemy는 Python에서 데이터베이스와 상호작용하기 위한 강력한 ORM(Object Relational Mapping) 라이브러리로, 이 코드는 데이터베이스 테이블의 스키마를 정의하고 관계를 설정하는 데 사용됩니다.

### `Answer` 클래스
`Answer` 클래스는 SQLAlchemy의 ORM을 사용하여 데이터베이스 테이블을 정의한 예입니다. 이 클래스는 `Base` 클래스를 상속받으며, `__tablename__` 속성을 통해 데이터베이스 테이블 이름을 `"answers"`로 지정합니다. 이 클래스는 다음과 같은 열을 정의합니다:
- `id`: 기본 키로 사용되는 정수형 열.
- `content`: 텍스트 데이터를 저장하며, `nullable=False`로 설정되어 있어 값이 반드시 있어야 합니다.
- `create_date`: 답변이 생성된 날짜와 시간을 저장하는 `DateTime` 열.
- `question_id`: 외래 키로, `"questions"` 테이블의 `id` 열을 참조합니다.
- `question`: `relationship`을 통해 `Question` 클래스와의 관계를 설정하며, `backref`를 사용하여 `answers`라는 역참조를 제공합니다.

### `relationship` 함수
`relationship` 함수는 SQLAlchemy에서 두 테이블 간의 관계를 정의하는 데 사용됩니다. 이 함수는 부모-자식 관계 또는 다대다 관계를 설정할 수 있습니다. 예를 들어, `Answer` 클래스에서 `question` 속성은 `Question` 클래스와의 관계를 나타냅니다. `backref` 매개변수를 통해 `Question` 클래스에서 `answers`라는 속성을 자동으로 생성하여 양방향 관계를 설정합니다. 이 함수는 다양한 매개변수를 지원하며, 관계의 로딩 전략(`lazy`), 삭제 시 동작(`cascade`), 정렬 순서(`order_by`) 등을 세부적으로 제어할 수 있습니다.

### `Column` 클래스
`Column` 클래스는 데이터베이스 테이블의 열을 나타냅니다. 이 클래스는 열의 이름, 데이터 타입, 기본값, 고유성, 인덱스 여부 등을 정의할 수 있습니다. 예를 들어, `id` 열은 `primary_key=True`로 설정되어 기본 키로 지정되며, `content` 열은 `nullable=False`로 설정되어 값이 반드시 필요합니다. 또한, `ForeignKey` 객체를 사용하여 다른 테이블의 열을 참조할 수 있습니다.

### `ForeignKey` 클래스
`ForeignKey` 클래스는 열 간의 외래 키 관계를 정의합니다. 예를 들어, `question_id` 열은 `ForeignKey("questions.id")`를 통해 `"questions"` 테이블의 `id` 열을 참조합니다. 이 클래스는 외래 키 제약 조건을 설정하며, `ondelete`, `onupdate`와 같은 매개변수를 통해 삭제 및 업데이트 시의 동작을 제어할 수 있습니다.

### 주요 동작
이 코드의 주요 동작은 데이터베이스 테이블과 열을 정의하고, 테이블 간의 관계를 설정하며, 외래 키를 통해 참조 무결성을 유지하는 것입니다. SQLAlchemy는 이러한 정의를 기반으로 데이터베이스 스키마를 생성하거나, ORM을 통해 Python 객체와 데이터베이스 간의 매핑을 제공합니다. 이를 통해 개발자는 SQL 쿼리를 직접 작성하지 않고도 데이터베이스와 상호작용할 수 있습니다.

# alembic
Alembic은 Python의 데이터베이스 마이그레이션 도구로, SQLAlchemy와 함께 사용하여 데이터베이스 스키마를 관리하고 버전 관리를 수행할 수 있습니다.

아래 파일들을 수정합니다.

[alembic.ini]
sqlalchemy.url = sqlite:///./myapi.db

[migrations/env.py]
import models

target_metadata = models.Base.metadata

### 마이그레이션 파일 생성
새로운 데이터베이스 변경 사항(예: 테이블 추가, 수정)을 반영하기 위해 마이그레이션 파일을 생성합니다.

--autogenerate 옵션은 SQLAlchemy의 Base.metadata를 기반으로 변경 사항을 자동으로 감지합니다.
생성된 파일은 versions/ 디렉토리에 저장됩니다.

alembic revision --autogenerate

### 리비전 파일 실행

alembic upgrade head

In [16]:
from models import Question, Answer
from datetime import datetime
from database import SessionLocal

db = SessionLocal()

In [17]:
# 데이터 넣기
q = Question(subject='pybo가 무엇인가요?', content='pybo에 대해서 알고 싶습니다.', create_date=datetime.now())
db.add(q)
db.commit()

q.id

1

In [18]:
q = Question(subject='FastAPI 모델 질문입니다.', content='id는 자동으로 생성되나요?', create_date=datetime.now())
db.add(q)
db.commit()

q.id

2

In [19]:
# 데이터 조회하기
db.query(Question).all()

[<models.Question at 0x10e60a450>, <models.Question at 0x10da1ac40>]

In [20]:
# filter 함수 이용한 질문
db.query(Question).filter(Question.id == 1).all()

In [21]:
# id는 유일한 값이므로 get 함수를 이용해 조회할 수도 있음
db.get(Question, 1)

In [22]:
# Like 검색
db.query(Question).filter(Question.subject.like('%FastAPI%')).all()

- 데이터를 조회하는 다양한 사용법
- https://docs.sqlalchemy.org/en/14/orm/query.html

In [23]:
# 데이터 수정하기
q = db.get(Question, 2)
q.subject = 'FastAPI Model Question'
db.commit()


In [24]:
# 데이터 삭제하기
q = db.get(Question, 1)
db.delete(q)
db.commit()

In [25]:
db.query(Question).all()

In [26]:
# 답변 데이터 저장하기
q = db.get(Question, 2)
a = Answer(question=q, content='네 자동으로 생성됩니다.', create_date=datetime.now())
db.add(a)
db.commit()

In [27]:
a.id

1

In [28]:
a.question

In [29]:
q.answers

# 라우터
- FastAPI가 요청받은 URL을 해석하여 그에 맞는 함수를 실행하여 그 결과를 리턴하는 행위를 말합니다.

In [5]:
from fastapi import APIRouter

from database import SessionLocal
from models import Question

router = APIRouter(
    prefix="/api/question",
)

@router.get("/list")
def question_list():
    db = SessionLocal()
    questions = db.query(Question).order_by(Question.create_date.desc()).all()
    db.close()
    return questions


In [6]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware 
from domain.question import question_router

app = FastAPI()

origins = [
    "http://localhost:5173",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(question_router.router)


In [8]:
import uvicorn
uvicorn.run(app)

INFO:     Started server process [1594]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50376 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50376 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50377 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50377 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50377 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1594]


### 테스트
- http://127.0.0.1:8000/docs

# 의존성 주입
### 데이터베이스 세션의 생성과 반환을 자동화하기
- db세션 객체를 생성한 후에 db.close()를 수행하지 않으면 SQLAlchemy가 사용하는 커넥션 풀에 db세션이 반환되지 않아 문제가 생깁니다.
- 프로그래밍에서 의존성 주입의 뜻은 필요한 기능을 선언하여 사용할 수 있다는 의미입니다.

## Contextmanager 사용

In [14]:
import contextlib

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./myapi.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

@contextlib.contextmanager
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


### 제너레이터란?
- https://wikidocs.net/134909

In [23]:
from fastapi import APIRouter

# from database import get_db
from models import Question

router = APIRouter(
    prefix="/api/question",
)

@router.get("/list")
def question_list():
    with get_db() as db:
        questions = db.query(Question).order_by(Question.create_date.desc()).all()            
    return questions


In [27]:
import uvicorn
uvicorn.run(app)

INFO:     Started server process [1594]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50639 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50639 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50639 - "GET /api/question/list HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1594]


### 테스트
- http://127.0.0.1:8000/docs

### Depends 사용하기
- FastAPI의 Depends를 사용하면 with문을 사용하는 것 보다 더 간단하게 사용할 수 있습니다.
- FastAPI는 제네레이터 기반 함수를 직접 지원하며, 자동으로 리소스 관리를 처리합니다.
- @contextlib.contextmanager를 사용하면 get_db 함수가 context_GeneratorContextManager 객체를 반환하게 되어 종속성 주입이 제대로 동작하지 않습니다.
- 어노테이션을 제거해야 합니다.

In [30]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./myapi.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [32]:
from fastapi import APIRouter, Depends
from sqlalchemy.orm import Session

# from database import get_db
from models import Question

router = APIRouter(
    prefix="/api/question",
)

@router.get("/list")
def question_list(db: Session = Depends(get_db)):
    questions = db.query(Question).order_by(Question.create_date.desc()).all()            
    return questions


### 파이썬 타입 어노테이션
- https://wikidocs.net/134883

In [33]:
import uvicorn
uvicorn.run(app)

INFO:     Started server process [1594]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50712 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50712 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50712 - "GET /api/question/list HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1594]


### 테스트
- http://127.0.0.1:8000/docs

# 스키마 (Pydantic)
- 모든 항목이 출력으로 리턴되지만 외부로 공개되면 안되는 출력항목이 있을 수 있습니다.
- 출력값이 정확한지 검증하고 싶을 수도 있습니다.

- 입출력 항목의 갯수와 타입을 설정
- 입출력 항목의 필수값 체크
- 입출력 항목의 데이터 검증

In [37]:
import datetime
from pydantic import BaseModel

class Question(BaseModel):
    id: int
    subject: str
    content: str
    create_date: datetime.datetime


In [38]:
from fastapi import APIRouter, Depends
from sqlalchemy.orm import Session

#from database import get_db
from domain.question import question_schema
from models import Question

router = APIRouter(
    prefix="/api/question",
)

@router.get("/list", response_model=list[question_schema.Question])
def question_list(db: Session = Depends(get_db)):
    questions = db.query(Question).order_by(Question.create_date.desc()).all()            
    return questions


In [39]:
from models import Question
from sqlalchemy.orm import Session

def get_question_list(db: Session):
    question_list = db.query(Question).order_by(Question.create_date.desc()).all()
    return question_list        
    

In [41]:
from fastapi import APIRouter, Depends
from sqlalchemy.orm import Session

# from database import get_db
from domain.question import question_schema, question_crud

router = APIRouter(
    prefix="/api/question",
)

@router.get("/list", response_model=list[question_schema.Question])
def question_list(db: Session = Depends(get_db)):
    questions = question_crud.get_question_list(db)            
    return questions


In [43]:
import uvicorn
uvicorn.run(app)

INFO:     Started server process [1594]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51157 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51157 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51157 - "GET /api/question/list HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1594]


### 테스트
- http://127.0.0.1:8000/docs

# SVELTE
- https://svelte.dev

- node.js 설치

- svelte project 
```shell
npm create vite@latest frontend -- --template svelte
cd ./frontend
npm install
```

- 자바스크립트 타입 체크 설정 끄기  
스펠트는 자바스크립트 타입을 체크하는 것이 디폴트로 설정되어 있습니다.  
타입스크립트를 사용하지 않으면 해당 설정을 끄고 진행하도록 합니다.  

```json
{
  "compilerOptions": {    
    /**
     * Typecheck JS in `.svelte` and `.js` files by default.
     * Disable this if you'd like to use dynamic types.
     */
    "checkJs": false
  },  
}
```

# svelte 서버 실행

```shell
npm run dev
```

### 프론트엔드 서버가 필요한 이유
- Svelte로 작성한 코드를 실시간으로 테스트 하려면, 브라우저가 인식할 수 있는 HTML, CSS, JS로 실시간 변환해주는 기능이 필요합니다.
- 코드를 빌드하면 순수 HTML/CSS/JS가 만들어지므로 FastAPI서버에 정적 파일 형태로 배포하면 되므로, Node.js 서버가 필요 없습니다.

In [ ]:
<script>
  let question_list = [];

  async function get_question_list() {
    const response = await fetch('http://127.0.0.1:8000/api/question/list')
    if (response.ok) {
      const data = await response.json();
      question_list = data;
    } else {
      console.error('Failed to fetch question list');
    }      
  }

  get_question_list();

</script>

<ul>
  {#each question_list as question}
    <li>{question.subject}</li>
  {/each}
</ul>

- question_list를 최초 빈 리스트인 []으로 초기값을 주지 않을 경우 비동기인 fetch함수가 요청하는 중에 HTML영역의 each문이 실행되고 question_list에 값이 없어 오류가 발생합니다.
- 반복문과 함께 사용할 경우 해당 값을 빈 리스트로 초기화 하는 것이 좋습니다.
- Svelte 특징중 하나 "Truly reactive"

# Svelte 필수 문법
## 분기문
```svelte
{#if 조건문1}
    <p>조건문1에 해당하면 실행</p>
{:else if 조건문2}
    <p>조건문2에 해당하면 실행</p>
{:else}
    <p>조건문1, 2 모두 해당하지 않으면 실행</p>
{/if}
```

## 반복문
```svelte
{#each list as item, index}
    <p>순서: {index} </p>
    <p>{item}</p>
{/each}
```

## 객체표시
```svelte
{객체}
{객체.속성}
```

- svelte SPA를 구현할때 svelte-spa-router를 사용하면 됩니다.
- svelte-spa-router는 Svelte로 작성된 싱글 페이지 애플리케이션(SPA)을 위한 라우터 라이브러리입니다. React Router나 Vue Router와 유사한 기능을 하며, URL 경로에 따라 다른 Svelte 컴포넌트를 렌더링하도록 도와줍니다.
- https://github.com/ItalyPaleAle/svelte-spa-router

- 설치
```bash
npm install svelte-spa-router
```


- 라우터 설정(예)
```svelte
<!-- App.svelte -->
<script>
  import Router from 'svelte-spa-router';
  import Home from './routes/Home.svelte';
  import About from './routes/About.svelte';

  const routes = {
    '/': Home,
    '/about': About
  };
</script>

<Router {routes} />
```

# 질문 상세 링크 추가하기
- a 태그에 use:link 속성을 사용하기 위해 link를 import를 했습니다.
```svelte
<script>
    import fastapi from "../lib/api"
    import { link } from 'svelte-spa-router'

    let question_list = []

    fastapi('get', '/api/question/list', {}, (json) => {
        question_list = json
    })
</script>

<ul>
    {#each question_list as question}
        <li><a use:link href="/detail/{question.id}">{question.subject}</a></li>
    {/each}
</ul>
```

## a 태그에 use:link를 사용하는 이유
- a 태그를 사용하는 일반적인 방법은 다음과 같습니다.
```html
<a href="/some-path">Some Path</a>
```
- 위와 같은 링크를 클릭하면 브라우저 주소창에 다음과 같이 표시됩니다.
```
http://127.0.0.1:5173/some-path
```
- 이번에는 다음처럼 a 태그에 use:link를 사용해봅니다.
```html
<a use:link href="/some-path">Some Path</a>
```
```
http://127.0.0.1:5173/#/some-path
```
- use:link속성을 사용하면 항상 /# 문자가 선행되도록 경로가 만들어집니다.
- 웹 페이지에서 어떤 경로가 /#으로 시작하면 브라우저는 이 경로를 하나의 페이지로 인식합니다.
- 즉 브라우저는 http://127.0.0.1:5173/#/a, http://127.0.0.1:5173/#/b 두개의 경로를 모두 동일한 페이지로 인식한다는 점입니다.
- 이러한 것을 해시 기반 라우팅이라고 합니다.

## 왜 해시 기반 라우팅을 사용하는 것을까요?
- /# 으로 시작할 경우 서버로 요청 보내지 않습니다. 
- 만약 클라이언트에만 존재하는 라우팅 주소는 서버에 없기 때문에 404 NotFound 오류가 발생하지 않습니다.

###  FastAPI 스키마 작성시 검증

In [4]:
from pydantic import BaseModel, field_validator

class AnswerCreate(BaseModel):
    content: str

    @field_validator('content')
    def not_empty(cls, v):
        if not v or not v.strip():
            raise ValueError('빈 값은 허용되지 않습니다.')
        return v

answer = AnswerCreate(content=' ')

ValidationError: 1 validation error for AnswerCreate
content
  Value error, 빈 값은 허용되지 않습니다. [type=value_error, input_value=' ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

### 입력 항목을 처리하는 스키마는 왜 필요할까?
- 답변 등록 API는 post방식이고 content라는 입력항목이 있습니다.
- 답변 등록 라우터에서 content 값을 읽기 위해서는 반드시 content항목을 포함하는 Pydantic 스키마를 통해 읽어야 합니다.
- get방식은 각각의 입력항목을 라우터 함수의 매개변수로 읽어야 하고, 그 외에는 Pydantic 스키마로만 읽어야 합니다.
- HTTP 프로토콜의 URL에 포함된 입력 값(URL parameter)은 라우터의 스키마가 아닌 매개변수로 읽는다. (Path Parameter, Query Parameter)
- HTTP 프로토콜의 Body에 포함된 입력 값(payload)은 Pydantic 스키마로 읽는다. (Request Body)

- 출력은 response_model을 사용하는 대신 status_code=status.HTTP_204_NO_CONTENT 를 사용했다. 이렇게 리턴할 응답이 없을 경우에는 응답코드 204를 리턴하여 "응답 없음"을 나타낼 수 있다.

In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy.orm import Session
from starlette import status

from database import get_db
from domain.answer import answer_schema, answer_crud
from domain.question import question_crud

router = APIRouter(
    prefix="/api/answer",
)

# 출력은 response_model을 사용하는 대신 status_code=status.HTTP_204_NO_CONTENT 를 사용했다. 이렇게 리턴할 응답이 없을 경우에는 응답코드 204를 리턴하여 "응답 없음"을 나타낼 수 있다.
@router.post("/create/{question_id}", status_code=status.HTTP_204_NO_CONTENT)
def answer_create(question_id: int,
                  _answer_create: answer_schema.AnswerCreate,
                  db: Session = Depends(get_db)):
    question = question_crud.get_question(db, question_id)
    if not question:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND,
                            detail="질문이 존재하지 않습니다.")
    answer_crud.create_answer(db, question, _answer_create)




## 스벨트 스토어
- 스벨트 스토어를 사용하면 변수 값을 전역적으로 저장할 수 있기 때문에 라우팅 되는 페이지에 상관없이 스토어에 저장된 변수를 사용할수 있습니다.
- 스토어 변수는 사용시 앞에 $을 붙여야 합니다.
- page -> $page

## 지속성 스토어(persistant store)
- localStorage를 사용하여 지속성을 지니게 합니다.

- $: get_question_list($page)의 의미는 page 값이 변경될 경우 get_question_list 함수도 다시 호출하라는 의미입니다.

## 시간 포맷을 사용할때
https://momentjs.com/

## 설치
```shell
npm install moment
```